In [1]:
import sys
sys.path.append('../modules')

from model import Resnet, Bottleneck
from utils import TinyImageNetDataset, get_preds, extract_CCI, diagonal_order
from cam import get_cam
from simba import SimBA

import cv2
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import numpy as np
import random
from pathlib import Path
import numpy as np 
import matplotlib.pyplot as plt

# Extract Correctly Classified Instances

In [2]:
transform = transforms.Compose([transforms.ToTensor()])
dataset = torchvision.datasets.CIFAR10(root='./data/cifar10', train=False, download=False, transform=transform)

In [2]:
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")

model_save_path = './model'
model_list = sorted([x for x in Path(model_save_path).glob('*_epoch_*.pt')], key = lambda x : int(x.stem.strip('model_epoch_')))

best_model_path = model_list[190]

net = Resnet(Bottleneck, [3, 4, 6, 3], num_classes=10).to(device)
net = torch.nn.DataParallel(net, device_ids=[4,5,6,7]).to(device)
net.load_state_dict(torch.load(best_model_path)['state'])

net.eval()

DataParallel(
  (module): Resnet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_st

In [4]:
dataset_name = 'cifar10'
num_data = 1000

In [5]:
extract_CCI(net, num_data, dataset_name, dataset)

../modules/utils.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds[idx], _ = get_preds(model, images[idx], dataset_name, batch_size=batch_size)


# Class Activation Map for CCI

In [3]:
CCI_data = torch.load('./data/cifar10_CCI.pth')
CCI_images, CCI_labels = CCI_data['images'], CCI_data['labels']

In [4]:
features_blobs = []

def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())
    
net._modules.get('module').layer4.register_forward_hook(hook_feature)

In [4]:
CCI_CAM = get_cam(net, features_blobs, CCI_images, 'cifar10')

NameError: name 'features_blobs' is not defined

In [6]:
torch.save({'CAM' : CCI_CAM}, './data/cifar10_CCI_CAM.pth')

# Simple Black-box Attack

In [4]:
import math

In [5]:
CCI_cam = torch.load('./data/cifar10_CCI_CAM.pth')['CAM']

In [8]:
image_size = 32
num_data = 1000
batch_size = 25
targeted = False
max_iters = 3*image_size*image_size
stride = 0 
epsilon = 0.2
order = 'rand'
log_every = 100
attacker = SimBA(net, 'cifar10', image_size)

In [9]:
N = int(math.floor(float(num_data) / float(batch_size)))

In [10]:
for i in range(N):
    upper = min((i + 1) * batch_size, num_data)
    images_batch = CCI_images[(i * batch_size):upper]
    labels_batch = CCI_labels[(i * batch_size):upper]
    adv, probs, succs, queries, l2_norms, linf_norms = attacker.simba_batch(
        images_batch, labels_batch, max_iters, stride, epsilon, order=order, targeted=targeted, log_every=log_every)
    
    print(f'{i}th batch attack is finished')
    
    if i == 0:
        all_adv = adv
        all_probs = probs
        all_succs = succs
        all_queries = queries
        all_l2_norms = l2_norms
        all_linf_norms = linf_norms
    else:
        all_adv = torch.cat([all_adv, adv], dim=0)
        all_probs = torch.cat([all_probs, probs], dim=0)
        all_succs = torch.cat([all_succs, succs], dim=0)
        all_queries = torch.cat([all_queries, queries], dim=0)
        all_l2_norms = torch.cat([all_l2_norms, l2_norms], dim=0)
        all_linf_norms = torch.cat([all_linf_norms, linf_norms], dim=0)

Iteration 99 : queries = 137.72000122070312 prob = 0.7330 progress = 40.00
Iteration 199 : queries = 219.8000030517578 prob = 0.6295 progress = 60.00
Iteration 299 : queries = 279.6400146484375 prob = 0.5913 progress = 72.00
Iteration 399 : queries = 335.6400146484375 prob = 0.5913 progress = 72.00
Iteration 499 : queries = 391.6400146484375 prob = 0.5913 progress = 72.00
Iteration 599 : queries = 447.6400146484375 prob = 0.5913 progress = 72.00
Iteration 699 : queries = 503.6400146484375 prob = 0.5913 progress = 72.00
Iteration 799 : queries = 559.6400146484375 prob = 0.5913 progress = 72.00
Iteration 899 : queries = 615.6400146484375 prob = 0.5913 progress = 72.00
Iteration 999 : queries = 671.6400146484375 prob = 0.5913 progress = 72.00
Iteration 1099 : queries = 727.6400146484375 prob = 0.5913 progress = 72.00
Iteration 1199 : queries = 783.6400146484375 prob = 0.5913 progress = 72.00
Iteration 1299 : queries = 839.6400146484375 prob = 0.5913 progress = 72.00
Iteration 1399 : queri

Iteration 1599 : queries = 1546.1600341796875 prob = 0.6920 progress = 56.00
Iteration 1699 : queries = 1634.1600341796875 prob = 0.6920 progress = 56.00
Iteration 1799 : queries = 1722.1600341796875 prob = 0.6920 progress = 56.00
Iteration 1899 : queries = 1810.1600341796875 prob = 0.6920 progress = 56.00
Iteration 1999 : queries = 1898.1600341796875 prob = 0.6920 progress = 56.00
Iteration 2099 : queries = 1986.1600341796875 prob = 0.6920 progress = 56.00
Iteration 2199 : queries = 2074.159912109375 prob = 0.6920 progress = 56.00
Iteration 2299 : queries = 2162.159912109375 prob = 0.6920 progress = 56.00
Iteration 2399 : queries = 2250.159912109375 prob = 0.6920 progress = 56.00
Iteration 2499 : queries = 2338.159912109375 prob = 0.6920 progress = 56.00
Iteration 2599 : queries = 2426.159912109375 prob = 0.6920 progress = 56.00
Iteration 2699 : queries = 2514.159912109375 prob = 0.6920 progress = 56.00
Iteration 2799 : queries = 2602.159912109375 prob = 0.6920 progress = 56.00
Iterat

Iteration 2999 : queries = 2469.39990234375 prob = 0.6627 progress = 60.00
Iteration 3071 : queries = 2527.0 prob = 0.6627 progress = 60.00
6th batch attack is finished
Iteration 99 : queries = 167.47999572753906 prob = 0.7867 progress = 32.00
Iteration 199 : queries = 269.32000732421875 prob = 0.6821 progress = 56.00
Iteration 299 : queries = 355.79998779296875 prob = 0.6804 progress = 56.00
Iteration 399 : queries = 438.20001220703125 prob = 0.6604 progress = 60.00
Iteration 499 : queries = 518.2000122070312 prob = 0.6604 progress = 60.00
Iteration 599 : queries = 598.1199951171875 prob = 0.6604 progress = 60.00
Iteration 699 : queries = 674.3599853515625 prob = 0.6374 progress = 64.00
Iteration 799 : queries = 746.3599853515625 prob = 0.6374 progress = 64.00
Iteration 899 : queries = 818.3200073242188 prob = 0.6374 progress = 64.00
Iteration 999 : queries = 888.3599853515625 prob = 0.6152 progress = 68.00
Iteration 1099 : queries = 952.3599853515625 prob = 0.6152 progress = 68.00
It

Iteration 1299 : queries = 1210.47998046875 prob = 0.6823 progress = 56.00
Iteration 1399 : queries = 1298.0799560546875 prob = 0.6823 progress = 56.00
Iteration 1499 : queries = 1384.239990234375 prob = 0.6598 progress = 60.00
Iteration 1599 : queries = 1464.239990234375 prob = 0.6598 progress = 60.00
Iteration 1699 : queries = 1544.239990234375 prob = 0.6598 progress = 60.00
Iteration 1799 : queries = 1624.239990234375 prob = 0.6598 progress = 60.00
Iteration 1899 : queries = 1704.239990234375 prob = 0.6598 progress = 60.00
Iteration 1999 : queries = 1784.239990234375 prob = 0.6598 progress = 60.00
Iteration 2099 : queries = 1864.239990234375 prob = 0.6598 progress = 60.00
Iteration 2199 : queries = 1944.239990234375 prob = 0.6598 progress = 60.00
Iteration 2299 : queries = 2024.239990234375 prob = 0.6598 progress = 60.00
Iteration 2399 : queries = 2104.239990234375 prob = 0.6598 progress = 60.00
Iteration 2499 : queries = 2184.239990234375 prob = 0.6598 progress = 60.00
Iteration 25

Iteration 2999 : queries = 2768.719970703125 prob = 0.6728 progress = 56.00
Iteration 3071 : queries = 2832.080078125 prob = 0.6728 progress = 56.00
13th batch attack is finished
Iteration 99 : queries = 145.39999389648438 prob = 0.7698 progress = 36.00
Iteration 199 : queries = 249.63999938964844 prob = 0.7011 progress = 52.00
Iteration 299 : queries = 344.32000732421875 prob = 0.7010 progress = 52.00
Iteration 399 : queries = 437.0400085449219 prob = 0.6795 progress = 56.00
Iteration 499 : queries = 525.0399780273438 prob = 0.6795 progress = 56.00
Iteration 599 : queries = 613.0399780273438 prob = 0.6795 progress = 56.00
Iteration 699 : queries = 701.0399780273438 prob = 0.6795 progress = 56.00
Iteration 799 : queries = 789.0399780273438 prob = 0.6795 progress = 56.00
Iteration 899 : queries = 877.0399780273438 prob = 0.6795 progress = 56.00
Iteration 999 : queries = 965.0399780273438 prob = 0.6795 progress = 56.00
Iteration 1099 : queries = 1053.0400390625 prob = 0.6795 progress = 5

...

Iteration 1099 : queries = 933.0 prob = 0.6392 progress = 64.00
Iteration 1199 : queries = 1005.0 prob = 0.6392 progress = 64.00
Iteration 1299 : queries = 1077.0 prob = 0.6392 progress = 64.00
Iteration 1399 : queries = 1149.0 prob = 0.6392 progress = 64.00
Iteration 1499 : queries = 1221.0 prob = 0.6392 progress = 64.00
Iteration 1599 : queries = 1293.0 prob = 0.6392 progress = 64.00
Iteration 1699 : queries = 1365.0 prob = 0.6392 progress = 64.00
Iteration 1799 : queries = 1437.0 prob = 0.6392 progress = 64.00
Iteration 1899 : queries = 1509.0 prob = 0.6392 progress = 64.00
Iteration 1999 : queries = 1581.0 prob = 0.6392 progress = 64.00
Iteration 2099 : queries = 1653.0 prob = 0.6392 progress = 64.00
Iteration 2199 : queries = 1725.0 prob = 0.6392 progress = 64.00
Iteration 2299 : queries = 1797.0 prob = 0.6392 progress = 64.00
Iteration 2399 : queries = 1869.0 prob = 0.6392 progress = 64.00
Iteration 2499 : queries = 1941.0 prob = 0.6392 progress = 64.00
Iteration 2599 : queries =

Iteration 2799 : queries = 1919.1600341796875 prob = 0.5953 progress = 68.00
Iteration 2899 : queries = 1983.1600341796875 prob = 0.5953 progress = 68.00
Iteration 2999 : queries = 2047.1600341796875 prob = 0.5953 progress = 68.00
Iteration 3071 : queries = 2093.239990234375 prob = 0.5953 progress = 68.00
27th batch attack is finished
Iteration 99 : queries = 137.63999938964844 prob = 0.6941 progress = 48.00
Iteration 199 : queries = 226.1199951171875 prob = 0.6409 progress = 56.00
Iteration 299 : queries = 309.3999938964844 prob = 0.6283 progress = 60.00
Iteration 399 : queries = 388.79998779296875 prob = 0.6277 progress = 60.00
Iteration 499 : queries = 463.6000061035156 prob = 0.6056 progress = 64.00
Iteration 599 : queries = 535.5999755859375 prob = 0.6056 progress = 64.00
Iteration 699 : queries = 607.5999755859375 prob = 0.6056 progress = 64.00
Iteration 799 : queries = 679.5999755859375 prob = 0.6056 progress = 64.00
Iteration 899 : queries = 751.5999755859375 prob = 0.6056 prog

Iteration 1199 : queries = 1107.1199951171875 prob = 0.6439 progress = 60.00
Iteration 1299 : queries = 1187.1199951171875 prob = 0.6439 progress = 60.00
Iteration 1399 : queries = 1267.1199951171875 prob = 0.6439 progress = 60.00
Iteration 1499 : queries = 1347.1199951171875 prob = 0.6439 progress = 60.00
Iteration 1599 : queries = 1427.1199951171875 prob = 0.6439 progress = 60.00
Iteration 1699 : queries = 1507.1199951171875 prob = 0.6439 progress = 60.00
Iteration 1799 : queries = 1587.1199951171875 prob = 0.6439 progress = 60.00
Iteration 1899 : queries = 1667.1199951171875 prob = 0.6439 progress = 60.00
Iteration 1999 : queries = 1747.1199951171875 prob = 0.6439 progress = 60.00
Iteration 2099 : queries = 1827.1199951171875 prob = 0.6439 progress = 60.00
Iteration 2199 : queries = 1907.1199951171875 prob = 0.6439 progress = 60.00
Iteration 2299 : queries = 1987.1199951171875 prob = 0.6439 progress = 60.00
Iteration 2399 : queries = 2067.1201171875 prob = 0.6439 progress = 60.00
It

Iteration 2599 : queries = 2327.60009765625 prob = 0.6764 progress = 56.00
Iteration 2699 : queries = 2415.60009765625 prob = 0.6764 progress = 56.00
Iteration 2799 : queries = 2503.60009765625 prob = 0.6764 progress = 56.00
Iteration 2899 : queries = 2591.60009765625 prob = 0.6764 progress = 56.00
Iteration 2999 : queries = 2679.60009765625 prob = 0.6764 progress = 56.00
Iteration 3071 : queries = 2742.9599609375 prob = 0.6764 progress = 56.00
34th batch attack is finished
Iteration 99 : queries = 151.72000122070312 prob = 0.6889 progress = 48.00
Iteration 199 : queries = 247.9199981689453 prob = 0.6776 progress = 52.00
Iteration 299 : queries = 342.3999938964844 prob = 0.6770 progress = 52.00
Iteration 399 : queries = 433.32000732421875 prob = 0.6554 progress = 56.00
Iteration 499 : queries = 520.5599975585938 prob = 0.6554 progress = 56.00
Iteration 599 : queries = 602.6799926757812 prob = 0.6349 progress = 60.00
Iteration 699 : queries = 682.6799926757812 prob = 0.6349 progress = 6

Iteration 899 : queries = 706.9199829101562 prob = 0.6396 progress = 64.00
Iteration 999 : queries = 778.9199829101562 prob = 0.6396 progress = 64.00
Iteration 1099 : queries = 850.9199829101562 prob = 0.6396 progress = 64.00
Iteration 1199 : queries = 922.9199829101562 prob = 0.6396 progress = 64.00
Iteration 1299 : queries = 994.9199829101562 prob = 0.6396 progress = 64.00
Iteration 1399 : queries = 1066.9200439453125 prob = 0.6396 progress = 64.00
Iteration 1499 : queries = 1138.9200439453125 prob = 0.6396 progress = 64.00
Iteration 1599 : queries = 1210.9200439453125 prob = 0.6396 progress = 64.00
Iteration 1699 : queries = 1282.9200439453125 prob = 0.6396 progress = 64.00
Iteration 1799 : queries = 1354.9200439453125 prob = 0.6396 progress = 64.00
Iteration 1899 : queries = 1426.9200439453125 prob = 0.6396 progress = 64.00
Iteration 1999 : queries = 1498.9200439453125 prob = 0.6396 progress = 64.00
Iteration 2099 : queries = 1570.9200439453125 prob = 0.6396 progress = 64.00
Iterat

In [11]:
save_path = './data/cifar10_SimBA_rand.pth'

torch.save({'adv': all_adv, 'probs': all_probs, 'succs': all_succs, 'queries': all_queries,'l2_norms': all_l2_norms, 'linf_norms': all_linf_norms}, save_path)

In [6]:
image_size = 32
num_data = 1000
batch_size = 25
targeted = False
max_iters = 3*image_size*image_size
stride = 0 
epsilon = 0.2
order = 'cam'
log_every = 100
attacker = SimBA(net, 'cifar10', image_size)

N = int(math.floor(float(num_data) / float(batch_size)))

In [7]:
for i in range(N):
    upper = min((i + 1) * batch_size, num_data)
    images_batch = CCI_images[(i * batch_size):upper]
    labels_batch = CCI_labels[(i * batch_size):upper]
    cam_batch = CCI_cam[(i * batch_size):upper]
    adv, probs, succs, queries, l2_norms, linf_norms = attacker.simba_batch(
        images_batch, labels_batch, max_iters, stride, epsilon, order=order, targeted=targeted, log_every=log_every, cam_batch=cam_batch)

    print(f'{i}th batch attack is finished')
    
    if i == 0:
        all_adv = adv
        all_probs = probs
        all_succs = succs
        all_queries = queries
        all_l2_norms = l2_norms
        all_linf_norms = linf_norms
    else:
        all_adv = torch.cat([all_adv, adv], dim=0)
        all_probs = torch.cat([all_probs, probs], dim=0)
        all_succs = torch.cat([all_succs, succs], dim=0)
        all_queries = torch.cat([all_queries, queries], dim=0)
        all_l2_norms = torch.cat([all_l2_norms, l2_norms], dim=0)
        all_linf_norms = torch.cat([all_linf_norms, linf_norms], dim=0)

Iteration 99 : queries = 156.8000030517578 prob = 0.7961 progress = 36.00
Iteration 199 : queries = 263.1600036621094 prob = 0.6871 progress = 52.00
Iteration 299 : queries = 343.0400085449219 prob = 0.6682 progress = 60.00
Iteration 399 : queries = 414.6000061035156 prob = 0.6411 progress = 64.00
Iteration 499 : queries = 478.67999267578125 prob = 0.6255 progress = 68.00
Iteration 599 : queries = 536.4000244140625 prob = 0.6075 progress = 72.00
Iteration 699 : queries = 592.4000244140625 prob = 0.6075 progress = 72.00
Iteration 799 : queries = 648.4000244140625 prob = 0.6075 progress = 72.00
Iteration 899 : queries = 704.4000244140625 prob = 0.6075 progress = 72.00
Iteration 999 : queries = 760.4000244140625 prob = 0.6075 progress = 72.00
Iteration 1099 : queries = 816.4000244140625 prob = 0.6075 progress = 72.00
Iteration 1199 : queries = 872.4000244140625 prob = 0.6075 progress = 72.00
Iteration 1299 : queries = 928.4000244140625 prob = 0.6075 progress = 72.00
Iteration 1399 : queri

Iteration 1599 : queries = 1553.9200439453125 prob = 0.7005 progress = 56.00
Iteration 1699 : queries = 1641.9200439453125 prob = 0.7005 progress = 56.00
Iteration 1799 : queries = 1729.9200439453125 prob = 0.7005 progress = 56.00
Iteration 1899 : queries = 1817.9200439453125 prob = 0.7005 progress = 56.00
Iteration 1999 : queries = 1905.9200439453125 prob = 0.7005 progress = 56.00
Iteration 2099 : queries = 1993.9200439453125 prob = 0.7005 progress = 56.00
Iteration 2199 : queries = 2081.919921875 prob = 0.7005 progress = 56.00
Iteration 2299 : queries = 2169.919921875 prob = 0.7005 progress = 56.00
Iteration 2399 : queries = 2257.919921875 prob = 0.7005 progress = 56.00
Iteration 2499 : queries = 2345.919921875 prob = 0.7005 progress = 56.00
Iteration 2599 : queries = 2433.919921875 prob = 0.7005 progress = 56.00
Iteration 2699 : queries = 2521.919921875 prob = 0.7005 progress = 56.00
Iteration 2799 : queries = 2609.919921875 prob = 0.7005 progress = 56.00
Iteration 2899 : queries = 

Iteration 99 : queries = 173.67999267578125 prob = 0.9239 progress = 12.00
Iteration 199 : queries = 318.2799987792969 prob = 0.7384 progress = 36.00
Iteration 299 : queries = 418.6000061035156 prob = 0.6927 progress = 52.00
Iteration 399 : queries = 500.4800109863281 prob = 0.6766 progress = 60.00
Iteration 499 : queries = 580.47998046875 prob = 0.6766 progress = 60.00
Iteration 599 : queries = 659.7999877929688 prob = 0.6766 progress = 60.00
Iteration 699 : queries = 736.1199951171875 prob = 0.6546 progress = 64.00
Iteration 799 : queries = 808.1199951171875 prob = 0.6546 progress = 64.00
Iteration 899 : queries = 880.1199951171875 prob = 0.6546 progress = 64.00
Iteration 999 : queries = 951.4000244140625 prob = 0.6545 progress = 64.00
Iteration 1099 : queries = 1018.760009765625 prob = 0.6341 progress = 68.00
Iteration 1199 : queries = 1082.760009765625 prob = 0.6341 progress = 68.00
Iteration 1299 : queries = 1146.760009765625 prob = 0.6341 progress = 68.00
Iteration 1399 : queries

Iteration 1599 : queries = 1299.6800537109375 prob = 0.6529 progress = 64.00
Iteration 1699 : queries = 1371.6800537109375 prob = 0.6529 progress = 64.00
Iteration 1799 : queries = 1443.6800537109375 prob = 0.6529 progress = 64.00
Iteration 1899 : queries = 1515.6800537109375 prob = 0.6529 progress = 64.00
Iteration 1999 : queries = 1587.6800537109375 prob = 0.6529 progress = 64.00
Iteration 2099 : queries = 1659.6800537109375 prob = 0.6529 progress = 64.00
Iteration 2199 : queries = 1731.6800537109375 prob = 0.6529 progress = 64.00
Iteration 2299 : queries = 1803.6800537109375 prob = 0.6529 progress = 64.00
Iteration 2399 : queries = 1875.6800537109375 prob = 0.6529 progress = 64.00
Iteration 2499 : queries = 1947.6800537109375 prob = 0.6529 progress = 64.00
Iteration 2599 : queries = 2019.6800537109375 prob = 0.6529 progress = 64.00
Iteration 2699 : queries = 2091.679931640625 prob = 0.6529 progress = 64.00
Iteration 2799 : queries = 2163.679931640625 prob = 0.6529 progress = 64.00
I

Iteration 3071 : queries = 2858.0400390625 prob = 0.6834 progress = 56.00
13th batch attack is finished
Iteration 99 : queries = 157.44000244140625 prob = 0.7861 progress = 36.00
Iteration 199 : queries = 266.32000732421875 prob = 0.7333 progress = 48.00
Iteration 299 : queries = 365.3999938964844 prob = 0.7116 progress = 52.00
Iteration 399 : queries = 461.3599853515625 prob = 0.7116 progress = 52.00
Iteration 499 : queries = 556.3200073242188 prob = 0.7109 progress = 52.00
Iteration 599 : queries = 648.6400146484375 prob = 0.6882 progress = 56.00
Iteration 699 : queries = 736.6400146484375 prob = 0.6882 progress = 56.00
Iteration 799 : queries = 824.6400146484375 prob = 0.6882 progress = 56.00
Iteration 899 : queries = 912.6400146484375 prob = 0.6882 progress = 56.00
Iteration 999 : queries = 1000.6400146484375 prob = 0.6882 progress = 56.00
Iteration 1099 : queries = 1088.6400146484375 prob = 0.6882 progress = 56.00
Iteration 1199 : queries = 1176.6400146484375 prob = 0.6882 progres

...

Iteration 1199 : queries = 1051.5999755859375 prob = 0.6475 progress = 60.00
Iteration 1299 : queries = 1130.6800537109375 prob = 0.6475 progress = 60.00
Iteration 1399 : queries = 1209.4000244140625 prob = 0.6467 progress = 60.00
Iteration 1499 : queries = 1288.52001953125 prob = 0.6269 progress = 64.00
Iteration 1599 : queries = 1360.52001953125 prob = 0.6269 progress = 64.00
Iteration 1699 : queries = 1432.52001953125 prob = 0.6269 progress = 64.00
Iteration 1799 : queries = 1504.52001953125 prob = 0.6269 progress = 64.00
Iteration 1899 : queries = 1576.52001953125 prob = 0.6269 progress = 64.00
Iteration 1999 : queries = 1648.52001953125 prob = 0.6269 progress = 64.00
Iteration 2099 : queries = 1720.52001953125 prob = 0.6269 progress = 64.00
Iteration 2199 : queries = 1792.52001953125 prob = 0.6269 progress = 64.00
Iteration 2299 : queries = 1864.52001953125 prob = 0.6269 progress = 64.00
Iteration 2399 : queries = 1936.52001953125 prob = 0.6269 progress = 64.00
Iteration 2499 : qu

Iteration 2699 : queries = 1862.1600341796875 prob = 0.5950 progress = 68.00
Iteration 2799 : queries = 1926.1600341796875 prob = 0.5950 progress = 68.00
Iteration 2899 : queries = 1990.1600341796875 prob = 0.5950 progress = 68.00
Iteration 2999 : queries = 2054.159912109375 prob = 0.5950 progress = 68.00
Iteration 3071 : queries = 2100.239990234375 prob = 0.5950 progress = 68.00
27th batch attack is finished
Iteration 99 : queries = 144.9199981689453 prob = 0.7715 progress = 40.00
Iteration 199 : queries = 252.83999633789062 prob = 0.6944 progress = 48.00
Iteration 299 : queries = 336.0799865722656 prob = 0.6533 progress = 60.00
Iteration 399 : queries = 412.5199890136719 prob = 0.6316 progress = 64.00
Iteration 499 : queries = 484.5199890136719 prob = 0.6316 progress = 64.00
Iteration 599 : queries = 556.52001953125 prob = 0.6316 progress = 64.00
Iteration 699 : queries = 628.52001953125 prob = 0.6316 progress = 64.00
Iteration 799 : queries = 700.52001953125 prob = 0.6316 progress =

Iteration 1099 : queries = 1015.52001953125 prob = 0.6652 progress = 60.00
Iteration 1199 : queries = 1095.52001953125 prob = 0.6652 progress = 60.00
Iteration 1299 : queries = 1175.52001953125 prob = 0.6652 progress = 60.00
Iteration 1399 : queries = 1255.52001953125 prob = 0.6652 progress = 60.00
Iteration 1499 : queries = 1335.52001953125 prob = 0.6652 progress = 60.00
Iteration 1599 : queries = 1415.52001953125 prob = 0.6652 progress = 60.00
Iteration 1699 : queries = 1495.52001953125 prob = 0.6652 progress = 60.00
Iteration 1799 : queries = 1575.52001953125 prob = 0.6652 progress = 60.00
Iteration 1899 : queries = 1655.52001953125 prob = 0.6652 progress = 60.00
Iteration 1999 : queries = 1735.52001953125 prob = 0.6652 progress = 60.00
Iteration 2099 : queries = 1815.52001953125 prob = 0.6652 progress = 60.00
Iteration 2199 : queries = 1895.52001953125 prob = 0.6652 progress = 60.00
Iteration 2299 : queries = 1975.52001953125 prob = 0.6652 progress = 60.00
Iteration 2399 : queries 

Iteration 2599 : queries = 2353.159912109375 prob = 0.6722 progress = 56.00
Iteration 2699 : queries = 2441.159912109375 prob = 0.6722 progress = 56.00
Iteration 2799 : queries = 2529.159912109375 prob = 0.6722 progress = 56.00
Iteration 2899 : queries = 2617.159912109375 prob = 0.6722 progress = 56.00
Iteration 2999 : queries = 2705.159912109375 prob = 0.6722 progress = 56.00
Iteration 3071 : queries = 2768.52001953125 prob = 0.6722 progress = 56.00
34th batch attack is finished
Iteration 99 : queries = 164.0 prob = 0.8063 progress = 24.00
Iteration 199 : queries = 274.6000061035156 prob = 0.7234 progress = 48.00
Iteration 299 : queries = 365.20001220703125 prob = 0.6841 progress = 56.00
Iteration 399 : queries = 447.6400146484375 prob = 0.6626 progress = 60.00
Iteration 499 : queries = 527.6400146484375 prob = 0.6626 progress = 60.00
Iteration 599 : queries = 607.6400146484375 prob = 0.6626 progress = 60.00
Iteration 699 : queries = 687.6400146484375 prob = 0.6626 progress = 60.00
It

Iteration 899 : queries = 726.3200073242188 prob = 0.6374 progress = 64.00
Iteration 999 : queries = 798.3200073242188 prob = 0.6374 progress = 64.00
Iteration 1099 : queries = 870.3200073242188 prob = 0.6374 progress = 64.00
Iteration 1199 : queries = 942.3200073242188 prob = 0.6374 progress = 64.00
Iteration 1299 : queries = 1014.3200073242188 prob = 0.6374 progress = 64.00
Iteration 1399 : queries = 1086.3199462890625 prob = 0.6374 progress = 64.00
Iteration 1499 : queries = 1158.3199462890625 prob = 0.6374 progress = 64.00
Iteration 1599 : queries = 1230.3199462890625 prob = 0.6374 progress = 64.00
Iteration 1699 : queries = 1302.3199462890625 prob = 0.6374 progress = 64.00
Iteration 1799 : queries = 1374.3199462890625 prob = 0.6374 progress = 64.00
Iteration 1899 : queries = 1446.3199462890625 prob = 0.6374 progress = 64.00
Iteration 1999 : queries = 1518.3199462890625 prob = 0.6374 progress = 64.00
Iteration 2099 : queries = 1590.3199462890625 prob = 0.6374 progress = 64.00
Itera

In [8]:
save_path = './data/cifar10_SimBA_cam.pth'

torch.save({'adv': all_adv, 'probs': all_probs, 'succs': all_succs, 'queries': all_queries,'l2_norms': all_l2_norms, 'linf_norms': all_linf_norms}, save_path)